In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd

train_users = pd.read_csv('../input/train_users_2.csv')
test_users = pd.read_csv('../input/test_users.csv')
sessions = pd.read_csv('../input/sessions.csv')

users = pd.concat([train_users, test_users], axis=0, ignore_index=True)


users.drop('date_first_booking', axis=1, inplace=True)

user_with_year_age_mask = users['age'] > 1900
users.loc[user_with_year_age_mask, 'age'] = 2015 - users.loc[user_with_year_age_mask, 'age']

users.loc[(users['age'] > 120) | (users['age'] < 18), 'age'] = -1
users['age'].fillna(-1, inplace=True)

bins = [-1, 20, 25, 30, 40, 50, 60, 75, 100, 120]
users['age_group'] = np.digitize(users['age'], bins, right=True)


list_browsers=["Arora","Avant Browser","Camino","CometBird","Comodo Dragon","Conkeror","CoolNovo","Crazy Browser","Epic","Flock","Google Earth","Googlebot","IBrowse","IceDragon","IceWeasel","Iron","Kindle Browser","Maxthon","Nintendo Browser","NetNewsWire","OmniWeb","Outlook 2007","Pale Moon","Palm Pre web browser","PS Vita browser","RockMelt","SeaMonkey","SiteKiosk","SlimBrowser","Sogou Explorer","Stainless","TenFourFox","TheWorld Browser","UC Browser","wOSBrowser","Yandex.Browser"]
for i in range(len(list_browsers)):
    users.loc[(users['first_browser'] == list_browsers[i]),'first_browser'] = 'Others'


users.loc[(users['first_browser'] == 'Chrome Mobile') ,'first_browser'] = 'Chrome'
users.loc[(users['first_browser'] == 'Chromium'),'first_browser'] = 'Chrome'
users.loc[(users['first_browser'] == 'Mobile Firefox') ,'first_browser'] = 'Firefox'
users.loc[(users['first_browser'] == 'Mozilla'),'first_browser'] = 'Firefox'
users.loc[(users['first_browser'] == 'IE Mobile'),'first_browser'] = 'IE'
users.loc[(users['first_browser'] == 'Mobile Safari') ,'first_browser'] = 'Safari'
users.loc[(users['first_browser'] == 'Opera Mobile') ,'first_browser'] = 'Opera'
users.loc[(users['first_browser'] == 'Opera Mini'),'first_browser'] = 'Opera'

users.loc[(users['affiliate_provider'] == 'facebook-open-graph') ,'affiliate_provider'] = 'facebook'

users.loc[(users['first_device_type'] == 'Other/Unknown') ,'first_device_type'] = 'Other'
users.loc[(users['first_device_type'] == 'Android Tablet') ,'first_device_type'] = 'Tablet'
users.loc[(users['first_device_type'] == 'iPad') ,'first_device_type'] = 'Tablet'
users.loc[(users['first_device_type'] == 'Android Phone') ,'first_device_type'] = 'Phones'
users.loc[(users['first_device_type'] == 'iPhone') ,'first_device_type'] = 'Phones'
users.loc[(users['first_device_type'] == 'SmartPhone (Other)') ,'first_device_type'] = 'Phones'
users.loc[(users['first_device_type'] == 'Desktop (Other)') ,'first_device_type'] = 'Desktop'
users.loc[(users['first_device_type'] == 'Mac Desktop') ,'first_device_type'] = 'Desktop'
users.loc[(users['first_device_type'] == 'Windows Desktop') ,'first_device_type'] = 'Desktop'

users['nans'] = np.sum([
    (users['age'] == -1),
    (users['gender'] == '-unknown-'),
    (users['language'] == '-unknown-'),
    (users['first_affiliate_tracked'] == 'untracked'),
    (users['first_browser'] == '-unknown-')
], axis=0)

users['date_account_created'] = pd.to_datetime(users['date_account_created'], errors='ignore')
users['date_first_active'] = pd.to_datetime(users['timestamp_first_active'], format='%Y%m%d%H%M%S')

date_account_created = pd.DatetimeIndex(users['date_account_created'])
date_first_active = pd.DatetimeIndex(users['date_first_active'])

users['day_account_created'] = date_account_created.day
users['weekday_account_created'] = date_account_created.weekday
users['week_account_created'] = date_account_created.week
users['month_account_created'] = date_account_created.month
users['year_account_created'] = date_account_created.year
users['day_first_active'] = date_first_active.day
users['weekday_first_active'] = date_first_active.weekday
users['week_first_active'] = date_first_active.week
users['month_first_active'] = date_first_active.month
users['year_first_active'] = date_first_active.year

users['time_lag'] = (date_account_created.values - date_first_active.values).astype(int)

drop_list = [
    'date_account_created',
    'date_first_active',
    'timestamp_first_active'
]

users.drop(drop_list, axis=1, inplace=True)

sessions.rename(columns = {'user_id': 'id'}, inplace=True)

action_count = sessions.groupby(['id', 'action'])['secs_elapsed'].agg(len).unstack()
action_type_count = sessions.groupby(['id', 'action_type'])['secs_elapsed'].agg(len).unstack()
action_detail_count = sessions.groupby(['id', 'action_detail'])['secs_elapsed'].agg(len).unstack()
device_type_sum = sessions.groupby(['id', 'device_type'])['secs_elapsed'].agg(sum).unstack()

sessions_data = pd.concat([action_count, action_type_count, action_detail_count, device_type_sum],axis=1)
sessions_data.columns = sessions_data.columns.map(lambda x: str(x) + '_count')

# session device_type
list_session_device_type=['Windows Desktop','Mac Desktop','Linux Desktop','Chromebook']
for i in range(len(list_session_device_type)):
    sessions.loc[(sessions['device_type'] == list_session_device_type[i]),'device_type'] = 'Desktop'

list_session_device_type=['Android Phone','Blackberry','iPhone','iPodtouch','Opera Phone','Windows Phone']
for i in range(len(list_session_device_type)):
    sessions.loc[(sessions['device_type'] == list_session_device_type[i]),'device_type'] = 'Phones'
    
list_session_device_type=['Android App Unknown Phone/Tablet','-unknown-']
for i in range(len(list_session_device_type)):
    sessions.loc[(sessions['device_type'] == list_session_device_type[i]),'device_type'] = 'Other'

## session_action
list_session_action=['agree_terms_check','agree_terms_uncheck']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'agree_terms'

list_session_action=['ajax_google_translate_description','ajax_google_translate_reviews']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'google_translate'

list_session_action=['ajax_payout_edit','ajax_payout_options_by_country']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'payout'
    
list_session_action=['ajax_photo_widget','ajax_photo_widget_form_iframe']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'photo_widget'
    
list_session_action=['ajax_referral_banner_experiment_type','ajax_referral_banner_type']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'banner'
    
list_session_action=['create','create_ach','create_multiple+create_paypal']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'create'

list_session_action=['complete','complete_redirect','complete_status']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'complete'    

list_session_action=['email_itinerary_colorbox','email_share','email_wishlist']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'email'     

list_session_action=['home_safety_landing','home_safety_terms']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'home_safety'  
    
list_session_action=['jumio_redirect','jumio_token']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'jumio'   

list_session_action=['message_to_host_change','message_to_host_focus']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'message_to_host'  

list_session_action=['qt_reply_v2','qt_with','qt2']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'qt' 

list_session_action=['update_cached','update_friends_display','update_hide_from_search_engines','update_notifications']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'update' 

list_session_action=['travel_plans_current','travel_plans_previous']
for i in range(len(list_session_action)):
    sessions.loc[(sessions['action'] == list_session_action[i]),'action'] = 'travel_plans'     

sessions.loc[(sessions['action'] == 'friends_new'),'action'] = 'friends' 
sessions.loc[(sessions['action'] == 'departments'),'action'] = 'department'
sessions.loc[(sessions['action'] == 'transaction_history_paginated'),'action'] = 'transaction_history'
sessions.loc[(sessions['action'] == 'edit_verification'),'action'] = 'edit'
sessions.loc[(sessions['action'] == 'kba_update'),'action'] = 'kba'   
sessions.loc[(sessions['action'] == 'listings'),'action'] = 'listing'   
sessions.loc[(sessions['action'] == 'multi_message'),'action'] = 'multi'  
sessions.loc[(sessions['action'] == 'reviews_new'),'action'] = 'reviews'  
sessions.loc[(sessions['action'] == 'similar_listings_v2'),'action'] = 'similar_listings' 
sessions.loc[(sessions['action'] == 'show_code'),'action'] = 'show' 
sessions.loc[(sessions['action'] == 'social_connections'),'action'] = 'social' 
sessions.loc[(sessions['action'] == 'terms_and_conditions'),'action'] = 'terms' 

#action_detail

list_coupon_session_action_detail=['apply_coupon','apply_coupon_click','apply_coupon_click_success','apply_coupon_error','coupon_code_click','coupon_field_focus']
for i in range(len(list_coupon_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_coupon_session_action_detail[i]),'sessions_action_detail'] = 'coupon'

list_cancel_session_action_detail=['cancellation_policies','cancellation_policy','cancellation_policy_click']
for i in range(len(list_cancel_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_cancel_session_action_detail[i]),'sessions_action_detail'] = 'cancel'

list_guest_session_action_detail=['guest_cancellation','guest_itinerary','guest_receipt']
for i in range(len(list_guest_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_guest_session_action_detail[i]),'sessions_action_detail'] = 'guest'
    
list_host_session_action_detail=['host_guarantee','host_home','host_refund_guest','host_respond','host_respond_page','host_standard_suspension']
for i in range(len(list_host_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_host_session_action_detail[i]),'sessions_action_detail'] = 'host'    
    
list_listing_session_action_detail=['listing_descriptions','listing_recommendations','listing_reviews','listing_reviews_page','manage_listing','view_listing']
for i in range(len(list_listing_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_listing_session_action_detail[i]),'sessions_action_detail'] = 'listing'       

     
list_message_session_action_detail=['lookup_message_thread','message_inbox','message_post','message_thread','message_to_host_change','message_to_host_focus']
for i in range(len(list_message_session_action_detail)):
    sessions.loc[(sessions['action_detail'] == list_message_session_action_detail[i]),'sessions_action_detail'] = 'message'       


# Most used device
sessions_data['most_used_device'] = sessions.groupby('id')['device_type'].max()

users = users.join(sessions_data, on='id')

secs_elapsed = sessions.groupby('id')['secs_elapsed']

secs_elapsed = secs_elapsed.agg(
    {
        'secs_elapsed_sum': np.sum,
        'secs_elapsed_mean': np.mean,
        'secs_elapsed_min': np.min,
        'secs_elapsed_max': np.max,
        'secs_elapsed_median': np.median,
        'secs_elapsed_std': np.std,
        'secs_elapsed_var': np.var,
        'day_pauses': lambda x: (x > 86400).sum(),
        'long_pauses': lambda x: (x > 300000).sum(),
        'short_pauses': lambda x: (x < 3600).sum(),
        'session_length' : np.count_nonzero
    }
)

users = users.join(secs_elapsed, on='id')


categorical_features = [
    'gender', 'signup_method', 'signup_flow', 'language',
    'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
    'signup_app', 'first_device_type', 'first_browser', 'most_used_device'
]
users = pd.get_dummies(users, columns=categorical_features)


users.set_index('id', inplace=True)
users.loc[train_users['id']].to_csv('train4_users.csv')
users.loc[test_users['id']].drop('country_destination', axis=1).to_csv('test4_users.csv')


#2eme partie 


import pandas as pd
import numpy as np
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import KFold

train_users = pd.read_csv('train4_users.csv')

train_users.fillna(-1, inplace=True)
y_train = train_users['country_destination']
train_users.drop(['country_destination', 'id'], axis=1, inplace=True)
x_train = train_users.values

label_encoder = LabelEncoder()
encoded_y_train = label_encoder.fit_transform(y_train)


train_data = xgboost.DMatrix(x_train, encoded_y_train)

param = {
    'max_depth': 4,
    'learning_rate': 0.01,
    'n_estimators': 5000,
    'objective': 'multi:softprob',
    'num_class': 12,
    'gamma': 0,
    'min_child_weight': 6,
    'max_delta_step': 0,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 1,
    'reg_alpha': 0.01,
    'reg_lambda': 1,
    'scale_pos_weight': 1,
    'base_score': 0.5,
    'missing': None,
    'silent': True,
    'nthread': 4,
    'seed': 27
}

num_round = 3
bst = xgboost.train(param, train_data, num_round)

import xgboost
test_users = pd.read_csv('test4_users.csv')
users_id = test_users['id']
test_users.drop(['id'], axis=1, inplace=True)
x_test = test_users.values
test_data = xgboost.DMatrix(x_test)

# make prediction
preds = bst.predict(test_data)


# 3eme partie 
# creer le fichier de submission

import csv
predictions = []
countries = []
header = ['id','country']
classes = list(set(zip(encoded_y_train,y_train)))
classes.sort(key=lambda x: x[0])
for c in range(len(classes)):
    countries.append(classes[c][1])
    
for p in range(len(preds)):
    zips = list(zip(preds[p],countries))
    zips.sort(key=lambda x: x[0],reverse=True)
    predictions.append(zips[0:5])

zips = list(zip(users_id, predictions))

with open("submission.csv", 'w',  newline='', encoding='utf-8') as my_airbnb_submission:
    wr = csv.DictWriter(my_airbnb_submission, fieldnames=header, lineterminator = '\r')
    wr.writeheader()
    for z in range(len(zips)):
        wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][0][1]})
        wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][1][1]})
        wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][2][1]})
        wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][3][1]})
        wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][4][1]})

#print(my_airbnb_submission)

In [ ]:
classes = list(set(zip(encoded_y_train,y_train)))
classes.sort(key=lambda x: x[0])
countries = []
for c in range(len(classes)):
    countries.append(classes[c][1])
print(countries)

In [ ]:
predictions = []
#countries = ['US', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL','DE', 'AU', 'NDF','other']
classes = list(set(zip(encoded_y_train,y_train)))
classes.sort(key=lambda x: x[0])
countries = []
for c in range(len(classes)):
    countries.append(classes[c][1])

for p in range(len(preds)):
    zips = list(zip(preds[p],countries))
    zips.sort(key=lambda x: x[0],reverse=True)
    predictions.append(zips[0:5])

zips = list(zip(users_id, predictions))
print(zips[0])
print(zips[0][0], zips[0][1][0][1])
print(zips[0][0], zips[0][1][1][1])
print(zips[0][0], zips[0][1][2][1])
print(zips[0][0], zips[0][1][3][1])
print(zips[0][0], zips[0][1][4][1])

In [ ]:
import csv
import pickle

predictions = []
countries = []
header = ['id','country']
classes = list(set(zip(encoded_y_train,y_train)))
classes.sort(key=lambda x: x[0])
for c in range(len(classes)):
    countries.append(classes[c][1])
    
for p in range(len(preds)):
    zips = list(zip(preds[p],countries))
    zips.sort(key=lambda x: x[0],reverse=True)
    predictions.append(zips[0:5])

zips = list(zip(users_id, predictions))
i = 0
with open("submission.csv", 'w',  newline='', encoding='utf-8') as my_airbnb_submission:
    wr = csv.DictWriter(my_airbnb_submission, fieldnames=header, lineterminator = ';')
    wr.writeheader()    
    for z in range(len(zips)):
        #if i < 310481:
        #    i += 5
        #else:
            wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][0][1]})
            wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][1][1]})
            wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][2][1]})
            wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][3][1]})
            wr.writerow({header[0]:zips[z][0], header[1]:zips[z][1][4][1]})

print(my_airbnb_submission)

In [ ]:
with open("submission.csv",'r') as submission:
    for r in submission:
        print(r)